<a href="https://colab.research.google.com/github/dongyeon2604-png/ManUtdAnalysis/blob/main/%EC%98%81%EC%96%B4_%EC%A0%95%EA%B7%9C%ED%99%94(NLTK).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import ast
import nltk
from nltk.stem import WordNetLemmatizer
from google.colab import files
import io

# 1. 필수 데이터 다운로드 (최초 1회)
print("필수 데이터 준비 중...")
nltk.download('wordnet')
nltk.download('omw-1.4')
print("준비 완료!\n")

# 2. 파일 업로드 (엑셀 파일 선택 가능)
print("▼ 아래 버튼을 눌러 엑셀(.xlsx) 파일을 업로드해주세요.")
uploaded = files.upload()

if not uploaded:
    print("파일이 업로드되지 않았습니다.")
else:
    # 3. 도구 준비 (표제어 추출기 & 유의어 사전)
    lemmatizer = WordNetLemmatizer()

    synonym_dict = {
        # [대회/리그]
        'ucl': 'champions_league', 'cl': 'champions_league', 'champions': 'champions_league',
        'epl': 'premier_league', 'pl': 'premier_league', 'prem': 'premier_league', 'league': 'premier_league',
        'fa': 'fa_cup',
        # [평가/행동]
        'sacked': 'sack', 'sacking': 'sack', 'fired': 'sack',
        'winning': 'win', 'won': 'win', 'winner': 'win',
        'losing': 'loss', 'lost': 'loss', 'defeat': 'loss',
        'tactical': 'tactic', 'tactics': 'tactic',
        'signing': 'sign', 'signed': 'sign', 'transfer': 'sign', 'bought': 'sign',
        # [감독/인물/팀]
        'saf': 'ferguson', 'sir': 'ferguson', 'alex': 'ferguson',
        'jose': 'mourinho', 'special': 'mourinho',
        'ole': 'solskjaer', 'ogs': 'solskjaer',
        'eth': 'ten_hag', 'erik': 'ten_hag',
        'lvg': 'van_gaal', 'louis': 'van_gaal',
        'city': 'man_city', 'pool': 'liverpool'
    }

    def normalize_english(text_data):
        try:
            # 엑셀은 리스트가 문자열로 저장되는 경우가 많아 변환 필요
            if isinstance(text_data, str) and text_data.startswith('['):
                words = ast.literal_eval(text_data)
            elif isinstance(text_data, list):
                words = text_data
            else:
                return []

            normalized = []
            for word in words:
                word = synonym_dict.get(word, word) # 1. 유의어 치환
                word = lemmatizer.lemmatize(word, pos='v') # 2. 동사 원형 복원
                word = lemmatizer.lemmatize(word, pos='n') # 3. 명사 단수형 복원
                normalized.append(word)
            return normalized
        except:
            return []

    print("\n작업을 시작합니다...\n")

    # 4. 업로드된 파일 처리
    for filename in uploaded.keys():
        try:
            # [핵심 수정] 확장자에 따라 읽는 방법 다르게 적용
            if filename.endswith('.xlsx'):
                print(f"[{filename}] 엑셀 파일로 읽습니다...")
                df = pd.read_excel(io.BytesIO(uploaded[filename]), engine='openpyxl')
            else:
                print(f"[{filename}] CSV 파일로 읽습니다...")
                df = pd.read_csv(io.BytesIO(uploaded[filename]))

            # 대상 컬럼 찾기
            target_col = 'Analyzed_Clean'
            if target_col not in df.columns:
                target_col = df.columns[-1] # 없으면 마지막 컬럼

            print(f" -> 정규화 적용 중 (대상 컬럼: {target_col})")

            # 정규화 함수 적용
            df['Normalized_Tokens'] = df[target_col].apply(normalize_english)

            # 5. 결과 저장 (엑셀로 저장)
            new_filename = "Norm_" + filename.replace(".csv", ".xlsx") # 확장자 엑셀로 통일
            if not new_filename.endswith('.xlsx'):
                new_filename += '.xlsx'

            df.to_excel(new_filename, index=False)
            print(f" -> 완료! 저장된 파일명: {new_filename}\n")

        except Exception as e:
            print(f" -> [오류] {filename} 처리 실패: {e}\n")

    print("모든 작업 완료! 왼쪽 폴더 아이콘에서 'Norm_' 파일을 다운로드하세요.")

필수 데이터 준비 중...
준비 완료!

▼ 아래 버튼을 눌러 엑셀(.xlsx) 파일을 업로드해주세요.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Saving Clean_분석결과_텐하흐 경질.xlsx to Clean_분석결과_텐하흐 경질.xlsx

작업을 시작합니다...

[Clean_분석결과_텐하흐 경질.xlsx] 엑셀 파일로 읽습니다...
 -> 정규화 적용 중 (대상 컬럼: Analyzed_Clean)
 -> 완료! 저장된 파일명: Norm_Clean_분석결과_텐하흐 경질.xlsx

모든 작업 완료! 왼쪽 폴더 아이콘에서 'Norm_' 파일을 다운로드하세요.
